In [1]:
from pulp import *
import pandas as pd
import numpy as np

In [2]:
probA=LpProblem("Problem A",LpMinimize)

In [4]:
df_warehouse_cost=pd.read_excel('./data/Exhibits.xlsx',sheet_name='Exhibit 7.3',header = 1).set_index('W/H')
df_demand=pd.read_excel('./data/Exhibits.xlsx',sheet_name='Exhibit 7.4',header = 1).set_index('Rep. no.') 
df_unit_cost=pd.read_excel('./data/Exhibits.xlsx',sheet_name='Exhibit 7.5',header = 1)
df_unit_cost = df_unit_cost.rename(index=str, columns={"Unnamed: 0": 'location'}).set_index('location')

In [18]:
#cost for each warehouse
#piecewise function
print(df_warehouse_cost.shape)
df_warehouse_cost.head(8)

(8, 5)


,Location,Fixed cost,Variable cost,Min (L),Max(U)
W/H,,,,,
1,Atlanta,2700,6,200,1750
2,Buffalo,2900,8,150,1250
3,Chicago,3500,9,250,2000
4,Cincinnati,2200,7,200,1500
5,Detroit,3300,8,200,1750
6,Pittsburgh,3000,8,200,1500
7,Richmond,2000,6,150,1000
8,St. Louis,1800,5,200,1500


In [25]:
#demands for each sales
print(df_demand.shape)
df_demand.head(10)

(22, 2)


,Location,Volume
Rep. no.,,
1,Atlanta,275
2,Birmingham,160
3,Buffalo,240
4,Charteston,260
5,Charlotte,135
6,Chattanooga,160
7,Chicago,400
8,Cincinnati,200
9,Cleveland,320


In [20]:
#cols:warehouse rows:sales
print(df_unit_cost.shape)
df_unit_cost.head(8)

(22, 8)


,Atlanta,Buffalo,Chicago,Cincinnati,Detroit,Pittsburgh,Richmond,St. Louis
location,,,,,,,,
Atlanta,1,13,16,11,15,11,8,12
Birmingham,4,15,15,12,16,13,11,10
Buffalo,13,1,10,8,4,4,7,13
Charteston,7,6,11,6,8,6,4,10
Charlotte,4,10,15,10,13,8,4,14
Chattanooga,2,11,14,9,13,9,7,11
Chicago,16,10,1,9,6,9,14,7
Cincinnati,9,6,7,3,6,6,7,7


In [9]:
#parameters
warehouse = df_warehouse_cost.Location.values
fixed_cost = df_warehouse_cost['Fixed cost'].values
fixed_cost = makeDict([warehouse],fixed_cost)

variable_cost = df_warehouse_cost['Variable cost'].values
variable_cost = makeDict([warehouse],variable_cost)

lowerB = df_warehouse_cost['Min (L)'].values
lowerB = makeDict([warehouse],lowerB)

upperB = df_warehouse_cost['Max(U)'].values
upperB = makeDict([warehouse],upperB)

representative = df_demand.Location.values
volume = df_demand.Volume.values
volume = makeDict([representative],volume)

unit_cost = df_unit_cost.values
unit_cost = makeDict([representative,warehouse],unit_cost)

In [10]:
#decision variables
shipQ = LpVariable.dicts("shipQ",(representative,warehouse),lowBound=0,cat="Continuous")
storage_below_min = LpVariable.dicts("storage_below_min",warehouse,lowBound=0,cat="Continuous")
storage_above_min = LpVariable.dicts("storage_above_min",warehouse,lowBound=0,cat="Continuous")
storage_decision = LpVariable.dicts("storage_decision",warehouse,cat="Binary")

In [11]:
#objective function
probA += lpSum([shipQ[a][b]*unit_cost[a][b] for a in representative for b in warehouse]) \
        + lpSum(variable_cost[x]*storage_above_min[x] for x in warehouse) \
        + lpSum(fixed_cost[y]*storage_decision[y] for y in warehouse)

In [12]:
#demand constraints
for a in representative:
    probA += lpSum(shipQ[a][b] for b in warehouse) == volume[a] 

#storage constraints
for x in warehouse:
    #upperBound
    probA += storage_below_min[x] + storage_above_min[x] <= upperB[x]*storage_decision[x]
    #lowerBound
    probA += storage_below_min[x] <= lowerB[x]*storage_decision[x]
    #ship_and_storage
    probA += storage_below_min[x] + storage_above_min[x] == lpSum(shipQ[y][x] for y in representative)

In [13]:
probA.writeLP("Hornby.lp")
probA.solve()
print("Status:",LpStatus[probA.status])

Status: Optimal


In [14]:
for v in probA.variables():
    print(v.name, "=", v.varValue,"\tReduced Cost =", v.dj)
print("Total cost =", value(probA.objective))

shipQ_Atlanta_Atlanta = 275.0 	Reduced Cost = 0.0
shipQ_Atlanta_Buffalo = 0.0 	Reduced Cost = 14.0
shipQ_Atlanta_Chicago = 0.0 	Reduced Cost = 23.0
shipQ_Atlanta_Cincinnati = 0.0 	Reduced Cost = 11.0
shipQ_Atlanta_Detroit = 0.0 	Reduced Cost = 19.0
shipQ_Atlanta_Pittsburgh = 0.0 	Reduced Cost = 11.0
shipQ_Atlanta_Richmond = 0.0 	Reduced Cost = 7.0
shipQ_Atlanta_St._Louis = 0.0 	Reduced Cost = 13.0
shipQ_Birmingham_Atlanta = 160.0 	Reduced Cost = 0.0
shipQ_Birmingham_Buffalo = 0.0 	Reduced Cost = 13.0
shipQ_Birmingham_Chicago = 0.0 	Reduced Cost = 19.0
shipQ_Birmingham_Cincinnati = 0.0 	Reduced Cost = 9.0
shipQ_Birmingham_Detroit = 0.0 	Reduced Cost = 17.0
shipQ_Birmingham_Pittsburgh = 0.0 	Reduced Cost = 10.0
shipQ_Birmingham_Richmond = 0.0 	Reduced Cost = 7.0
shipQ_Birmingham_St._Louis = 0.0 	Reduced Cost = 8.0
shipQ_Buffalo_Atlanta = 0.0 	Reduced Cost = 10.0
shipQ_Buffalo_Buffalo = 240.0 	Reduced Cost = 0.0
shipQ_Buffalo_Chicago = 0.0 	Reduced Cost = 15.0
shipQ_Buffalo_Cincinnati = 0

storage_above_min_Cincinnati = 660.0 	Reduced Cost = 0.0
storage_above_min_Detroit = 0.0 	Reduced Cost = 8.0
storage_above_min_Pittsburgh = 0.0 	Reduced Cost = 8.0
storage_above_min_Richmond = 410.0 	Reduced Cost = 0.0
storage_above_min_St._Louis = 1300.0 	Reduced Cost = 0.0
storage_below_min_Atlanta = 200.0 	Reduced Cost = 0.0
storage_below_min_Buffalo = 150.0 	Reduced Cost = 0.0
storage_below_min_Chicago = 0.0 	Reduced Cost = 0.0
storage_below_min_Cincinnati = 200.0 	Reduced Cost = 0.0
storage_below_min_Detroit = 0.0 	Reduced Cost = 0.0
storage_below_min_Pittsburgh = 0.0 	Reduced Cost = 0.0
storage_below_min_Richmond = 150.0 	Reduced Cost = 0.0
storage_below_min_St._Louis = 200.0 	Reduced Cost = 0.0
storage_decision_Atlanta = 1.0 	Reduced Cost = 1500.0
storage_decision_Buffalo = 1.0 	Reduced Cost = 1700.0
storage_decision_Chicago = 0.0 	Reduced Cost = -24500.0
storage_decision_Cincinnati = 1.0 	Reduced Cost = 800.0
storage_decision_Detroit = 0.0 	Reduced Cost = -15950.0
storage_decis

In [15]:
output = []
for x in representative:
    temp=[]
    for y in warehouse:
        temp.append(shipQ[x][y].varValue)
    output.append(temp)
output_df = pd.DataFrame(output,index=representative, columns=warehouse)
output_df

,Atlanta,Buffalo,Chicago,Cincinnati,Detroit,Pittsburgh,Richmond,St. Louis
Atlanta,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Birmingham,160.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Buffalo,0.0,240.0,0.0,0.0,0.0,0.0,0.0,0.0
Charteston,0.0,0.0,0.0,0.0,0.0,0.0,260.0,0.0
Charlotte,135.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chattanooga,160.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicago,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0
Cincinnati,0.0,0.0,0.0,200.0,0.0,0.0,0.0,0.0
Cleveland,0.0,320.0,0.0,0.0,0.0,0.0,0.0,0.0
Columbus,0.0,0.0,0.0,220.0,0.0,0.0,0.0,0.0


In [16]:
output = []
for x in warehouse:
    output.append(storage_below_min[x].varValue+storage_above_min[x].varValue)
output1_df = pd.DataFrame(output,index=warehouse)
output1_df.rename(index=str, columns= {0: 'volume'}, inplace = True)
output1_df

,volume
Atlanta,1065.0
Buffalo,1090.0
Chicago,0.0
Cincinnati,860.0
Detroit,0.0
Pittsburgh,0.0
Richmond,560.0
St. Louis,1500.0
